In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp, to_date, datediff, current_date, lit, concat_ws, year, floor, current_timestamp
from pyspark.sql.types import StringType

# Initialize SparkSession if not already available
spark = SparkSession.builder.appName("Users Transform").getOrCreate()

# Define Unity Catalog catalog and schema names
# IMPORTANT: Replace 'your_catalog' and 'your_schema' with your actual desired Unity Catalog catalog and schema names.
catalog_name = "main_catalog"
schema_name = "silver_users_schema"

# Create catalog and schema if they don't exist
spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}")
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {schema_name}")
spark.sql(f"USE SCHEMA {schema_name}")

# Define the Unity Catalog input and output table names
input_table_name = "main_catalog.bronze_users_schema.users_raw"
output_table_name = f"{catalog_name}.{schema_name}.users_transformed"

# Read the raw data from the Unity Catalog table
df = spark.read.table(input_table_name)

# Perform transformations
# Assuming the raw DataFrame 'df' has columns like:
# user_id, first_name, last_name, email, date_of_birth, registration_date

transformed_df = df.withColumn(
    # 1. Create a full_name column
    "full_name", concat_ws(" ", col("first_name"), col("last_name"))
).withColumn(
    # 2. Convert string dates to proper date/timestamp types
    "registration_ts", to_timestamp("registration_date", "yyyy-MM-dd HH:mm:ss")
).withColumn(
    "birth_date", to_date("date_of_birth", "yyyy-MM-dd")
).withColumn(
    # 3. Calculate user's current age
    "age", floor(datediff(current_date(), col("birth_date")) / 365.25)
).withColumn(
    # 4. Calculate user's tenure in days
    "days_as_member", datediff(current_date(), col("registration_ts"))
).withColumn(
    # 5. Add a metadata column for load timestamp
    "_load_timestamp", current_timestamp()
).select(
    # 6. Select and reorder columns for the final silver table
    col("user_id"),
    col("full_name"),
    col("email"),
    col("age"),
    col("days_as_member"),
    col("registration_ts"),
    col("_load_timestamp")
)

# Write the transformed data to the Unity Catalog silver table
transformed_df.write.mode("overwrite").option("mergeSchema", "true").saveAsTable(output_table_name)

print(f"Successfully transformed data and saved to {output_table_name}")

# Display a sample of the transformed data
display(spark.read.table(output_table_name))